In [103]:
import torch
import torchvision
from torchvision import transforms
from torchvision.datasets import ImageFolder
import numpy
import torch.nn.functional as func

In [117]:
import pickle
import matplotlib.pyplot as plt

f = open("MNIST", "rb")
data = pickle.load(f)
f.close()
print("Training Set Size:", len(data["train"]))
print("Testing Set Size:", len(data["test"]))


#plt.imshow(data["train"][0]["image"], cmap="Greys")
#plt.colorbar()
#plt.show()

Training Set Size: 60000
Testing Set Size: 10000


In [84]:
type(data["train"][0]["image"])

numpy.ndarray

In [124]:
# Convert data into tensors
x_train = []
for i in range(len(data["train"])):
    x_train.append(torch.FloatTensor(data["train"][i]["image"]))

x_train = torch.stack(x_train)

#print(type(x_train))
#print(type(x_train[0]))
#print(x_train.shape)

y_train = []
for j in range(len(data["train"])):
    y_train.append(data["train"][j]["label"])

y_train = torch.Tensor(y_train)
#print(type(y_train))
#print(type(y_train[0]))
#print(y_train.shape)

x_test = []
for a in range(len(data["test"])):
    x_test.append(torch.FloatTensor(data["test"][a]["image"]))

x_test = torch.stack(x_test)

#print(type(x_train))
#print(type(x_train[0]))
#print(x_train.shape)

y_test = []
for b in range(len(data["test"])):
    y_test.append(data["test"][b]["label"])

y_test = torch.Tensor(y_test)


In [120]:
y_train = func.one_hot(y_train.long(), num_classes=10)
y_train.shape

torch.Size([60000, 10])

In [121]:
y_train

tensor([[0, 0, 0,  ..., 0, 0, 0],
        [1, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 1, 0]])

In [87]:
x_train[0].shape

torch.Size([28, 28])

In [127]:
from torch.utils.data import Dataset, DataLoader

class TensorDataset(Dataset):
    def __init__(self, x, y):
        # Init
        self.x = x
        self.y = y

        # Normalize between 0 and 1
        self.x = self.x / 255.

        #One Hot Encoding
        self.y = func.one_hot(self.y.long(), num_classes=10).to(float)
    
    def __getitem__(self, index):
        return self.x[index], self.y[index]
    
    def __len__(self):
        return self.x.shape[0]

In [128]:
trainingDS = TensorDataset(x_train, y_train)
testingDS = TensorDataset(x_test, y_test)

In [65]:
#transformation = transforms.Compose([transforms.Normalize(0.1307, 0.3081),(0.1307, 0.3081)])

In [66]:
dataset = TensorDataset(train_images, train_labels, test_images, test_labels)

ValueError: Found array with dim 3. StandardScaler expected <= 2.

In [55]:
loader = DataLoader(dataset, batch_size=60, shuffle=True, num_workers=8)

In [34]:
counter = 0
for epoch in [1,2,3]:
    for batch, label in loader:
        counter +=1
        if counter == 1:
            print(batch, label)
        print(batch.size(), label.size())

tensor([[[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        ...,

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0., 

In [57]:
dataset[0]
#dataset.target[0]

(tensor([[  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
            0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
            0.,   0.,   0.,   0.],
         [  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
            0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
            0.,   0.,   0.,   0.],
         [  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
            0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
            0.,   0.,   0.,   0.],
         [  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
            0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
            0.,   0.,   0.,   0.],
         [  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
            0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
            0.,   0.,   0.,   0.],
     

In [24]:
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = torch.nn.Conv2d(1,6,5)
        self.pool = torch.nn.MaxPool2d(2,2)
        self.conv2 = torch.nn.Conv2d(6, 16, 5)
        self.fc1 = torch.nn.Linear(16*4*4, 120)
        self.fc2 = torch.nn.Linear(120, 84)
        self.fc3 = torch.nn.Linear(84,10)


    def forward(self, x):
        x = self.pool(torch.nn.functional.relu(self.conv1(x)))
        x = self.pool(torch.nn.functional.relu(self.conv2(x)))
        x = x.view(-1, 16 * 4 * 4)
        x = torch.nn.functional.relu(self.fc1(x))
        x = torch.nn.functional.relu(self.fc2(x))
        x = self.fc3(x)
        return x